In [1]:
# See [https://www.kaggle.com/code/camillahorne/poker-data-extraction]

# Generate input (x) and output (Y) data from the Poker Hold'Em Games dataset
# For training and validating a machine learning model

The game state will be defined by the following parameters:

Game Data
- Players
- Pot
- Button

Player Data
- Chips
- Bet
- Hand
    - Card 1
    - Card 2

Opponent Data x8
- Chips
- Action (Fold | Call | Raise | All-In)
- Bet

In [7]:
# Parse training data into x and Y data np.arrays
# Keep track of game state
# On each new game clear the game state
# For each move, update the game state
# Whenever the player makes a move concatenate the game state and the player move to the training data

In [8]:
# Save the training data into two csv files
# Parse the .csv files into np arrays with:
# np.genfromtxt('file.csv', delimiter=',')

In [2]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import json
import glob

In [3]:
def GetGamesData():
    result = []
    
    for dirname, _, filenames in os.walk('*/Data/'):
        for filename in filenames:
            file = open(os.path.join(dirname, filename), 'r')
            content = file.read()

            result += re.findall(r'Game started at: (.+)\nGame ID: (\d+).+\(Hold\'em\)\nSeat (\d+) is the button\n((?:Seat .+\n)+)([^$]+?)------ Summary ------\n(Pot: .+?)\n(?:(Board: .+?)\n)?([^$]+?)Game ended at: (.+)', content)

            file.close()
    
    print ("Total of", len(result), "games")
    
    return (result)

games = GetGamesData()

Total of 0 games


In [4]:
datetimeFormat = '%Y/%m/%d %H:%M:%S'
seatRe = re.compile(r"Seat (\d+): (.+) \((\d+(?:\.\d+)?)\)")
betRe = re.compile(r"Pot: (\d+(?:\.\d+)?).+Rake (\d+(?:\.\d+)?)(?:.+JP fee (\d+(?:\.\d+)?))?")
boardRe = re.compile(r"\[([^]]*)\]")
playerNameRe = re.compile(r"Player ([^ ]+)")
actionRe = re.compile(r"Player .+ ([^ ]+)s? \((\d+(?:\.\d+)?)\)")
endingNumberRe = re.compile(r".+\((\d+(?:\.\d+)?)\)")
cardsRe = re.compile(r".*\[([^\]]+)\]")
winningRe = re.compile(r"Player .+?(?: \[(.+)\])?\)?\. ?Bets: (\d+(?:\.\d+)?)\. Collects: (\d+(?:\.\d+)?)\.(?: Wins: (\d+(?:\.\d+)?)\.)?(?: Loses: (\d+(?:\.\d+)?)\.)?")

In [5]:
def GetPlayerName(line):
    match = playerNameRe.search(line)
    
    if match is None:
        return ""
    
    return match.group(1)

def GetDuration(startTime, endTime):
    tStart = datetime.strptime(startTime, datetimeFormat)
    tEnd = datetime.strptime(endTime, datetimeFormat)
    tDelta = tEnd - tStart
    return int(tDelta.total_seconds())

def GetAsFloat(value):
    return float(value if value else 0)

In [12]:
cards = {'1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'J':11, 'Q':12, 'K':13, 'A':14}
suits = {'h' : 1, 'd' : 2, 'c' : 3, 's' : 4}
actions = {'Fold' : 1, 'Call' : 2, 'Raise' : 3, 'All-In' : 4}

def ClearDataframe():
    global gameState
    # max 9 players at the table including our player
    gameState = pd.DataFrame({'numPlayers': [], 'button': [], 'pot': [], 'position': [], 'chips': [], 'stake': [], 'card1': [], 'card2': [], 'P1_chips': [], 'P1_action': [], 'P1_bet': [], 'P2_chips': [], 'P2_action': [], 'P2_bet': [], 'P3_chips': [], 'P3_action': [], 'P3_bet': [], 'P4_chips': [], 'P4_action': [], 'P4_bet': [], 'P5_chips': [], 'P5_action': [], 'P5_bet': [], 'P6_chips': [], 'P6_action': [], 'P6_bet': [], 'P7_chips': [], 'P7_action': [], 'P7_bet': [], 'P8_chips': [], 'P8_action': [], 'P8_bet': [], 'P9_chips': [], 'P9_action': [], 'P9_bet': []})
    
    global action
    action = pd.DataFrame({'action': [], 'bet': []})

In [9]:
tmpPath = 'tmp'
gameStateFileName = 'game_state.csv'
actionFileName = 'action.csv'

def CleanEverything():
    for file in glob.glob(f'*.csv'):
        os.remove(file)

def PrepareTmpFolder(delete = False):
    ClearDataframes()
    
    if not os.path.exists(tmpPath):
        os.makedirs(tmpPath)
    elif delete:
        files = os.listdir(tmpPath)
        for file in files:
            file_path = os.path.join(tmpPath, file)
            os.remove(file_path)

def SaveTmpDatasets(index):
    currentIndex = str(index).rjust(5, '0')
    gameState.to_csv(f'{tmpPath}/{currentIndex}-{gameStateFileName}', index=False)
    action.to_csv(f'{tmpPath}/{currentIndex}-{actionFileName}', index=False)

The following requires updating and refactoring for our purposes:

In [ ]:
def GetCurrentPlayer(playerDict, name):
    if name in playerDict:
        return playerDict[name]

    # Beware: Hack to handle space in username. Hopefully this is enough.
    playerNames = list(playerDict.keys())

    for playerName in playerNames:
        if playerName.startswith(name):
            return playerDict[playerName]


def ExtractGameData(currentGame):
    startTime = currentGame[0]
    gameId = int(currentGame[1])
    button = int(currentGame[2])
    seats = currentGame[3]
    gameActions = currentGame[4]
    bet = currentGame[5]
    board = currentGame[6]
    winnings = currentGame[7]
    endTime = currentGame[8]


    # Extract players
    players = seatRe.findall(seats)
    
    playerDict = {}
    seatsInPlay = []
    
    for player in players:
        playerSeat = int(player[0])
        playerName = player[1]
        playerChips = float(player[2])

        seatsInPlay += [playerSeat]
        playerDict[playerName] = {'seat': playerSeat, 'chips': playerChips, 'name': playerName, 'cards': []}


    # Extract actions
    actionsList = gameActions.split("\n")
    currentState = "PREFLOP"
    
    for action in actionsList:
        if action == "":
            continue
        if action.startswith("*** "):
            stateGroup = re.search(r'\*{3} ([^ ]+) \*{3}', action)
            currentState = stateGroup.group(1)
            continue
        if action.startswith("Uncalled bet"):
            continue
        
        currentPlayerName = GetPlayerName(action)
        currentPlayer = GetCurrentPlayer(playerDict, currentPlayerName)
        
        if currentPlayer is None:
            # game 30837 contains lines where the player name is missing
            continue

        if "has small blind" in action:
            smallBlindValue = float(endingNumberRe.search(action).group(1))
            gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'SB', smallBlindValue, gameId]
            continue
        if "has big blind" in action:
            bigBlindValue = float(endingNumberRe.search(action).group(1))
            gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'BB', bigBlindValue, gameId]
            continue
        if action.endswith(" received a card."):
            continue
        if " received card: [" in action:
            playerDict[playerName]["cards"] += cardsRe.search(action).group(1).split(" ")
            continue

        if action.endswith("is timed out."):
            if (currentPlayer):
                gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'timed out', 0, gameId]
            continue
        if action.endswith("sitting out"):
            gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'sitting out', 0, gameId]
            continue
        if action.endswith("wait BB"):
            gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'wait BB', 0, gameId]
            continue
        if action.endswith("mucks cards"):
            gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'muck', 0, gameId]
            continue
        if action.endswith("folds"):
            gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'fold', 0, gameId]
            continue
        if action.endswith("checks"):
            gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], 'check', 0, gameId]
            continue

        actionMatch = actionRe.match(action)
        gameActionsDf.loc[len(gameActionsDf)] = [currentState, currentPlayer['name'], currentPlayer["seat"], actionMatch.group(1), float(actionMatch.group(2)), gameId]


    # Extract bet values
    betGroups = betRe.search(bet).groups()
    gamePot = GetAsFloat(betGroups[0])
    gameRake = GetAsFloat(betGroups[1])
    gameFee = GetAsFloat(betGroups[2])


    # Extract board values
    boardCards = []
    if board:
        boardMatch = boardRe.search(board)
        boardCards = boardMatch.group(1).split(" ")


    # Extract winnings
    for winning in re.sub("\*", "", winnings).split("\n"):
        if winning == "":
            continue

        winningMatches = winningRe.search(winning)
        winningCards = winningMatches.group(1)

        currentPlayerName = GetPlayerName(winning)
        currentPlayer = GetCurrentPlayer(playerDict, currentPlayerName)
        
        playerGameStatsDf.loc[len(playerGameStatsDf)] = [currentPlayer['name'], winningCards.split(" ") if winningCards else None, GetAsFloat(winningMatches.group(2)), GetAsFloat(winningMatches.group(3)), GetAsFloat(winningMatches.group(4)), GetAsFloat(winningMatches.group(5)), gameId]


    # Extract duration
    duration = GetDuration(startTime, endTime)
    

    # Save result
    playersJson = json.dumps(list(playerDict.values()))
    seatsJson = json.dumps(seatsInPlay)
    boardJson = json.dumps(boardCards)
    gameDf.loc[len(gameDf)] = [gameId, duration, button, seatsJson, playersJson, currentState, gamePot, gameRake, gameFee, boardJson]

In [ ]:
PrepareTmpFolder(True)
totalCount = len(games)

for index, currentGame in enumerate(games):
    ExtractGameData(currentGame)

    if index % 1000 == 0 and index > 0:
        SaveTmpDatasets(int(index / 1000))
        print (f'{index}/{totalCount} done')

print ('All done')

In [ ]:
tmpFileNames = [gameDataframeFileName, gameActionsDataframeFileName, playerGameStatsDataframeFileName]

for tmpFileName in tmpFileNames:
    joined_files = os.path.join(tmpPath, f'*-{tmpFileName}')
    joined_list = sorted(glob.glob(joined_files))

    df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
    df.to_csv(tmpFileName, index=False)

In [ ]:
# Delete temporary data
if os.path.exists('/kaggle/working/tmp'):
    files = os.listdir('/kaggle/working/tmp')
    for file in files:
        file_path = os.path.join('/kaggle/working/tmp', file)
        os.remove(file_path)
    os.rmdir('/kaggle/working/tmp')